In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


ConnectionRefusedError: [Errno 111] Connection refused

In [8]:
import time
from pyspark.sql.functions import col, when, rand, expr
from pyspark.sql import SparkSession

# Initialize Spark session with memory optimizations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS")

# Debugging: Print total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))  # Ensure at least 1 row is updated

# Define batch size
batch_size = 500  # Reduced batch size to optimize memory usage
num_batches = max(1, (num_rows // batch_size) + (1 if num_rows % batch_size else 0))  # Ensure at least 1 batch

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

st = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")

    # Select a batch of unique row IDs without collecting to the driver
    sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(min(batch_size, num_rows))

    # Merge updates instead of overwriting partitions
    updated_batch = df.alias("source").join(sampled_df.alias("updates"), "extra_col_0", "inner") \
        .withColumn("extra_col_1", col("extra_col_1") + 10)

    # Optimize writing using append mode or merge
    updated_batch.repartition(100).writeTo("demo.nyc.taxis_10M_50COLUMNS").append()

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end = time.time() - st
print(f"\nTotal update time for {num_batches} batches: {end:.2f} sec")


Total rows in table: 10004500


Enter update percentage (e.g., 1 for 1%):  1


Updating 100045 rows (~1.0%) in 201 batches...
Processing batch 1/201...


25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:17:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Batch 1/201 updated.
Processing batch 2/201...


...

25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 09:42:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Batch 9/201 updated.
Processing batch 10/201...


[6058.891s][warning][gc,alloc] Executor task launch worker for task 231.0 in stage 305.0 (TID 10072): Retried waiting for GCLocker too often allocating 1048578 words


25/03/04 09:45:04 ERROR Executor: Exception in task 231.0 in stage 305.0 (TID 10072)
java.lang.OutOfMemoryError: Java heap space
25/03/04 09:45:04 WARN TaskSetManager: Lost task 231.0 in stage 305.0 (TID 10072) (5e0ed98abed9 executor driver): java.lang.OutOfMemoryError: Java heap space

25/03/04 09:45:04 ERROR TaskSetManager: Task 231 in stage 305.0 failed 1 times; aborting job
25/03/04 09:45:04 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 231.0 in stage 305.0 (TID 10072),5,main]
java.lang.OutOfMemoryError: Java heap space
25/03/04 09:45:04 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-244b373c-a8e3-4302-95fa-49829b358285/15/temp_shuffle_ed1e3a84-2ee4-4891-badc-0569d2bb0869
25/03/04 09:45:04 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-244b373c-a8e3-4302-95fa-49829b358285/3f/temp_shuffle_6327d217-3c86-4b4b-9eeb-a525b91abbb9
25/03/04 09:45:04 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-2

Py4JJavaError: An error occurred while calling o497.append.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 231 in stage 305.0 failed 1 times, most recent failure: Lost task 231.0 in stage 305.0 (TID 10072) (5e0ed98abed9 executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAG

In [2]:
import time
from pyspark.sql.functions import expr, rand
from pyspark.sql import SparkSession

# Initialize Spark session with memory optimizations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.memory.fraction", "0.7") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.autoBroadcastJoinThreshold", "50MB") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS")

# Debugging: Print total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))  # Ensure at least 1 row is updated

# Define batch size dynamically
batch_size = min(500, num_rows // 10)  # Adjust batch size based on total updates
num_batches = max(1, (num_rows // batch_size) + (1 if num_rows % batch_size else 0))  # Ensure at least 1 batch

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

st = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")
    
    # Select a batch of unique row IDs without collecting to the driver
    sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(min(batch_size, num_rows))
    
    # Use MERGE INTO for efficient updates
    spark.sql(f'''
        MERGE INTO demo.nyc.taxis_10M_50COLUMNS AS source
        USING (SELECT extra_col_0 FROM sampled_df) AS updates
        ON source.extra_col_0 = updates.extra_col_0
        WHEN MATCHED THEN
        UPDATE SET source.extra_col_1 = source.extra_col_1 + 10
    ''')

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end = time.time() - st
print(f"\nTotal update time for {num_batches} batches: {end:.2f} sec")


25/03/04 09:56:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Total rows in table: 10009002


Enter update percentage (e.g., 1 for 1%):  1


Updating 100090 rows (~1.0%) in 201 batches...
Processing batch 1/201...


25/03/04 09:56:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `sampled_df` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 39;
'MergeIntoTable ('source.extra_col_0 = 'updates.extra_col_0), [updateaction(None, assignment('source.extra_col_1, ('source.extra_col_1 + 10)))]
:- SubqueryAlias source
:  +- SubqueryAlias demo.nyc.taxis_10M_50COLUMNS
:     +- RelationV2[extra_col_0#165, extra_col_1#166, extra_col_2#167, extra_col_3#168, extra_col_4#169, extra_col_5#170, extra_col_6#171, extra_col_7#172, extra_col_8#173, extra_col_9#174, extra_col_10#175, extra_col_11#176, extra_col_12#177, extra_col_13#178, extra_col_14#179, extra_col_15#180, extra_col_16#181, extra_col_17#182, extra_col_18#183, extra_col_19#184, extra_col_20#185, extr

In [3]:
import time
from pyspark.sql.functions import expr, rand
from pyspark.sql import SparkSession

# Initialize Spark session with memory optimizations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.memory.fraction", "0.7") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.autoBroadcastJoinThreshold", "50MB") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS")

# Debugging: Print total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))  # Ensure at least 1 row is updated

# Define batch size dynamically
batch_size = min(500, num_rows // 10)  # Adjust batch size based on total updates
num_batches = max(1, (num_rows // batch_size) + (1 if num_rows % batch_size else 0))  # Ensure at least 1 batch

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

st = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")

    # Select a batch of unique row IDs without collecting to the driver
    sampled_df = df.select("extra_col_0").distinct().orderBy(rand()).limit(min(batch_size, num_rows))

    # Register the sampled DataFrame as a temporary view
    sampled_df.createOrReplaceTempView("sampled_temp_view")

    # Use MERGE INTO for efficient updates
    spark.sql(f'''
        MERGE INTO demo.nyc.taxis_10M_50COLUMNS AS source
        USING sampled_temp_view AS updates
        ON source.extra_col_0 = updates.extra_col_0
        WHEN MATCHED THEN
        UPDATE SET source.extra_col_1 = source.extra_col_1 + 10
    ''')

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end = time.time() - st
print(f"\nTotal update time for {num_batches} batches: {end:.2f} sec")


Total rows in table: 10009002


Enter update percentage (e.g., 1 for 1%):  1


Updating 100090 rows (~1.0%) in 201 batches...
Processing batch 1/201...


AnalysisException: [INVALID_NON_DETERMINISTIC_EXPRESSIONS] The operator expects a deterministic expression, but the actual expression is "(extra_col_0 = extra_col_0)", "exists(extra_col_0)".; line 2 pos 8;
ReplaceData
:  +- Filter (outer(extra_col_0#380) = extra_col_0#490)
:     +- SubqueryAlias updates
:        +- SubqueryAlias sampled_temp_view
:           +- View (`sampled_temp_view`, [extra_col_0#490])
:              +- GlobalLimit 500
:                 +- LocalLimit 500
:                    +- Project [extra_col_0#490]
:                       +- Sort [_nondeterministic#379 ASC NULLS FIRST], true
:                          +- Project [extra_col_0#490, rand(-1231214445441557296) AS _nondeterministic#379]
:                             +- Deduplicate [extra_col_0#490]
:                                +- Project [extra_col_0#490]
:                                   +- SubqueryAlias demo.nyc.taxis_10M_50COLUMNS
:                                      +- RelationV2[extra_col_0#490, extra_

In [12]:
from pyspark.sql import SparkSession
import time

# Initialize Spark session with Iceberg support
spark = SparkSession.builder \
    .appName("Iceberg Merge") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "hadoop") \
    .config("spark.sql.catalog.demo.warehouse", "s3://your-bucket/warehouse") \
    .getOrCreate()

# Load the Iceberg table
df = spark.read.format("iceberg").load("demo.nyc.taxis_10M_50COLUMNS")

# Get user input for update percentage
update_percentage = float(input("Enter the percentage of records to update (0-100): ")) / 100

# Calculate batch size
num_rows = df.count()
batch_size = int(num_rows * update_percentage)
st = time.time()

# Select distinct values of extra_col_0 for sampling
sampled_df = df.select("extra_col_0").distinct().orderBy("extra_col_0").limit(batch_size)

# Register as a temporary view
sampled_df.createOrReplaceTempView("sampled_temp_view")

# Count records before update for verification
pre_update_count = df.filter(df.extra_col_0.isin([row.extra_col_0 for row in sampled_df.collect()])).count()

# Perform MERGE INTO operation
spark.sql('''
    MERGE INTO demo.nyc.taxis_10M_50COLUMNS AS source
    USING sampled_temp_view AS updates
    ON source.extra_col_0 = updates.extra_col_0
    WHEN MATCHED THEN
    UPDATE SET source.extra_col_1 = source.extra_col_1 + 10
''')

# Count records after update
post_update_count = df.filter(df.extra_col_0.isin([row.extra_col_0 for row in sampled_df.collect()])).count()

# Calculate the number of records updated
updated_count = post_update_count - pre_update_count

print(f"Number of records updated: {updated_count}")
print("Update completed successfully.")
print(f"Time taken: {time.time() - st} seconds")


Enter the percentage of records to update (0-100):  1


25/03/04 10:14:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:14:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/04 10:16:43 WARN DAGScheduler: Broadcasting large task bin

[1663.798s][warning][gc,alloc] refresh progress: Retried waiting for GCLocker too often allocating 330 words


25/03/04 10:20:42 ERROR Utils: uncaught error in thread Spark Context Cleaner, stopping SparkContext
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ContextCleaner$$Lambda$865/0x00000008406ee840.get$Lambda(Unknown Source)
	at java.base/java.lang.invoke.DirectMethodHandle$Holder.invokeStatic(DirectMethodHandle$Holder)
	at java.base/java.lang.invoke.Invokers$Holder.linkToTargetMethod(Invokers$Holder)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$1(ContextCleaner.scala:195)
	at org.apache.spark.ContextCleaner$$Lambda$781/0x00000008406a8c40.apply$mcV$sp(Unknown Source)
	at org.apache.spark.util.Utils$.tryOrStopSparkContext(Utils.scala:1356)
	at org.apache.spark.ContextCleaner.org$apache$spark$ContextCleaner$$keepCleaning(ContextCleaner.scala:189)
	at org.apache.spark.ContextCleaner$$anon$1.run(ContextCleaner.scala:79)
25/03/04 10:20:43 ERROR Utils: throw uncaught fatal error in thread Spark Context Cleaner
java.lang.OutOfMemoryError: Java heap space
	at org.apa

[1680.296s][warning][gc,alloc] Executor task launch worker for task 3.0 in stage 49.0 (TID 3212): Retried waiting for GCLocker too often allocating 32291 words


Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: Java heap space
25/03/04 10:21:08 WARN Utils: Suppressing exception in finally: Cannot return unfinished footer.
java.lang.IllegalStateException: Cannot return unfinished footer.
	at org.apache.iceberg.shaded.org.apache.parquet.Preconditions.checkState(Preconditions.java:156)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.ParquetFileWriter.getFooter(ParquetFileWriter.java:1386)
	at org.apache.iceberg.parquet.ParquetWriter.metrics(ParquetWriter.java:144)
	at org.apache.iceberg.io.DataWriter.close(DataWriter.java:90)
	at org.apache.iceberg.io.RollingFileWriter.closeCurrentWriter(RollingFileWriter.java:122)
	at org.apache.iceberg.io.RollingFileWriter.close(RollingFileWriter.java:147)
	at org.apache.iceberg.io.RollingDataWriter.close(RollingDataWriter.java:32)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.close(SparkWrite.java:747)
	at org.apache.spark.sql.execution.dataso

Py4JJavaError: An error occurred while calling o39.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 49.0 failed 1 times, most recent failure: Lost task 1.0 in stage 49.0 (TID 3210) (5e0ed98abed9 executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.s

25/03/04 10:21:09 ERROR Executor: Exception in task 0.0 in stage 49.0 (TID 3209)
java.lang.OutOfMemoryError: Java heap space
25/03/04 10:21:09 ERROR SparkUncaughtExceptionHandler: [Container in shutdown] Uncaught exception in thread Thread[Executor task launch worker for task 0.0 in stage 49.0 (TID 3209),5,main]
java.lang.OutOfMemoryError: Java heap space
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539

In [13]:
from pyspark.sql import SparkSession
import time

# Initialize Spark session with Iceberg support
spark = SparkSession.builder \
    .appName("Iceberg Merge") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "hadoop") \
    .config("spark.sql.catalog.demo.warehouse", "s3://your-bucket/warehouse") \
    .getOrCreate()

# Load the Iceberg table
df = spark.read.format("iceberg").load("demo.nyc.taxis_10M_50COLUMNS")

# Get user input for update percentage
update_percentage = float(input("Enter the percentage of records to update (0-100): ")) / 100

# Calculate batch size
num_rows = df.count()
batch_size = int(num_rows * update_percentage)
st = time.time()

# Select distinct values of extra_col_0 for sampling
sampled_df = df.select("extra_col_0").distinct().orderBy("extra_col_0").limit(batch_size)

# Register as a temporary view
sampled_df.createOrReplaceTempView("sampled_temp_view")

# Perform MERGE INTO operation
spark.sql('''
    MERGE INTO demo.nyc.taxis_10M_50COLUMNS AS source
    USING sampled_temp_view AS updates
    ON source.extra_col_0 = updates.extra_col_0
    WHEN MATCHED THEN
    UPDATE SET source.extra_col_1 = source.extra_col_1 + 10
''')

print("Update completed successfully.")
print(f"Time taken: {time.time() - st} seconds")


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: SparkSession$ does not exist in the JVM

In [5]:
df = spark.sql("SELECT snapshot_id, made_current_at FROM demo.nyc.taxis_10M_50COLUMNS.history ORDER BY made_current_at DESC")

df.show()

+-------------------+--------------------+
|        snapshot_id|     made_current_at|
+-------------------+--------------------+
| 105668051919482591|2025-03-04 10:00:...|
|6814201661817395182|2025-03-04 09:42:...|
|6521404266517949867|2025-03-04 09:39:...|
|7604542288563918334|2025-03-04 09:36:...|
| 552092269728169030|2025-03-04 09:33:...|
|2209669498964280618|2025-03-04 09:30:...|
|7674319906980595297|2025-03-04 09:27:...|
|5468151910761078557|2025-03-04 09:23:...|
|7947044984117739091|2025-03-04 09:20:...|
|5673072808774441768|2025-03-04 09:17:...|
|1470626163957892397|2025-03-04 09:09:...|
|7160248633578306637|2025-03-04 09:06:...|
|5174434347034670474|2025-03-04 08:47:...|
|1794110359935204866|2025-03-04 08:44:...|
|6937317397142763324|2025-03-04 08:41:...|
|3089841895765010104|2025-03-04 08:38:...|
|7127818708494625786|2025-03-04 08:34:...|
|2707916599315632908|2025-03-04 08:30:...|
| 497229154627925394|2025-03-04 08:25:...|
|7776288292023736999|2025-03-04 08:13:...|
+----------

In [8]:
df=spark.sql("SELECT * FROM demo.nyc.taxis_10M_50COLUMNS.snapshots ORDER BY committed_at desc")
df.show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-03-04 10:00:...| 105668051919482591|6814201661817395182|overwrite|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:42:...|6814201661817395182|6521404266517949867|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:39:...|6521404266517949867|7604542288563918334|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:36:...|7604542288563918334| 552092269728169030|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:33:...| 552092269728169030|2209669498964280618|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:30:...|2209669498964280618|7674319906980595297|   append|s3://war